In [ ]:
!pip install textblob
!python -m textblob.download_corpora

In [1]:
import json
import string
import re
import pymongo

# from pymongo import MongoClient
from textblob import TextBlob
from textblob import Word
from nltk.corpus import stopwords
from collections import Counter
from pprint import pprint

In [2]:
stopwords = stopwords.words('English')
stoplist = string.punctuation.split() + stopwords

# print(stoplist)

In [ ]:
# 從現有檔案讀入文章
with open('news.news.json',encoding='utf8') as f:
    texts = json.load(f)

contents = texts['content']

# pprint(contents)
# type(contents)

In [10]:
# Get connection to MongoDB@10.120.28.55
try:
    client = pymongo.MongoClient('10.120.28.55', 27017)
    db = client['news']
    collection_read = db['news']
    collection_write = db['word_dict']
    
except:
    print('Cannot connect to MongoDB server!')

In [21]:
articles = collection_read.find({"source":"abc"})

In [22]:
for article in articles:
    contents = article['content']
    art_id = article['_id']
    wordList = getWordList(art_id, contents)
    updateWord_dictCollection(wordList)

Update to MongoDB successfully!
Update to MongoDB successfully!
Update to MongoDB successfully!
Update to MongoDB successfully!
Update to MongoDB successfully!
Update to MongoDB successfully!
Update to MongoDB successfully!
Update to MongoDB successfully!
Update to MongoDB successfully!
Update to MongoDB successfully!
Update to MongoDB successfully!
Update to MongoDB successfully!
Update to MongoDB successfully!
Update to MongoDB successfully!
Update to MongoDB successfully!
Update to MongoDB successfully!
Update to MongoDB successfully!
Update to MongoDB successfully!
Update to MongoDB successfully!
Update to MongoDB successfully!
Update to MongoDB successfully!
Update to MongoDB successfully!
Update to MongoDB successfully!
Update to MongoDB successfully!
Update to MongoDB successfully!
Update to MongoDB successfully!
Update to MongoDB successfully!
Update to MongoDB successfully!
Update to MongoDB successfully!
Update to MongoDB successfully!
Update to MongoDB successfully!
Update t

In [ ]:
words_dict = {}
newdict = updateWordDict(collection_read.find_one(), words_dict)

In [ ]:
pprint(newdict)

In [ ]:
# 斷詞、計次、紀錄來源段落

def getWordDict(collectionFind, words_dict):
    
    contents = collectionFind['content']
    art_id = collectionFind['_id']

    for numPara in range(len(contents)):
        para_id = str(numPara + 1)
        para_wds = str(TextBlob(contents[para_id]).words.lower().singularize())

        match = re.findall('[a-z]+', para_wds)

        for word in match:
            if word not in stoplist and len(word)>2:

                wd = Word(word).lemmatize("v")

                if wd not in words_dict:
                    src_list = set(['%s_%s'%(art_id, para_id)])
                    word_info = {}
                    word_info['count'] = 1
                    word_info['src'] = src_list
                    words_dict[wd] = word_info
                else:
                    words_dict[wd]['count'] += 1
                    words_dict[wd]['src'].add('%s_%s'%(art_id, para_id))
            
    return words_dict

# pprint(words_dict)

In [7]:
# 從 MongoDB取得一篇文章，並取得該篇文章的wordList

article = collection_read.find_one()
contents = article['content']
art_id = article['_id']
wordList = getWordList(art_id, contents)

In [16]:
def getWordList(art_id, contents):
    
    wordList = []

    for numPara in range(len(contents)):
        para_id = str(numPara + 1)
        para_wds = str(TextBlob(contents[para_id]).words.lower().singularize())

        match = re.findall('[a-z]+', para_wds)

        for word in match:
            if word not in stoplist and len(word)>2:
                finalWord = Word(word).lemmatize("v")

                word_info = {}
                word_info['_id'] = finalWord
                word_info['count'] = 1
                word_info['src'] = '%s_%s'%(art_id, para_id)
                wordList.append(word_info)
    
    return wordList

# pprint(wordList)

In [17]:
def updateWord_dictCollection(wordList):
    for obj in wordList:
        if collection_write.find({'_id':obj['_id']},{'Available':True}):
            collection_write.find_one_and_update(
                {'_id':obj['_id']},
                {'$inc':{'count':1}, '$addToSet':{'src':obj['src']}},
                upsert=True)
        else:
            collection_write.insert_one(obj)
    print('Update to MongoDB successfully!') 

In [9]:
# 更新資料庫 word_dict cllection
updateWord_dictCollection(wordList)

Update to MongoDB successfully!


In [ ]:
para = tb(content['1'])
para2 = tb(content['2'])
# give a tag for each word
para.tags
# get noun phrases from text
para.noun_phrases
# get single word
para.words
# count
para.word_counts
# # get single sentences
# print(para2.sentences)
# # objective(0.0), subjective(1.0)
# para2.sentences[2].sentiment.polarity
# test = 'I like python so much!'
# tb(test).sentiment.polarity